전처리
1. 6번 이상 평가(=시청)된 item 데이터만 남기기
2. item id remapping
    - 학습을 위해 기존 item ID를 1부터 재할당한다.
    - 242: 1,302: 2, 377: 3, 51: 4, 346: 5, 474: 6, ... ,
    - 예외 케이스는 0으로 할당한다.
3. user id remappding
4. get user session
    - user1:      [1, 526, 378, 520, 431, 818, 381, 330, 547, 84...

In [ ]:
import pandas as pd

In [ ]:
# 데이터 불러오기
file_path = '/content/drive/MyDrive/논문 구현/FPMC/u.data'
names = ['user', 'item', 'rating', 'timestamps']
data = pd.read_csv(file_path, header=None, sep='\t', names=names)

In [ ]:
# 6번 이상 평가(=시청)된 item 데이터만 남기기 위한 전처리 작업
p = data.groupby('item')['user'].count().reset_index().rename(columns={'user':'item_count'}) # 각 item이 몇 번 평가(=시청)되었는지 count함.
data = pd.merge(data, p, how='left', on='item') # count한 결과를 원본 data와 결합.
data = data[data['item_count'] > 5].drop(['item_count'], axis=1) # 6번 이상 평가(=시청)된 데이터만 남김. 그리고 item_count 열은 이제 필요없으므로 drop함.

In [ ]:
# item id remapping 하기
item_unique = data['item'].unique().tolist() # unique한 item list 생성
item_map = dict(zip(item_unique, range(1, len(item_unique) + 1)))
item_map[-1] = 0 # 예외 케이스는 0으로 할당함. 예를 들어, 데이터셋에 결측치를 -1로 표기하거나, 특정 조건을 만족하지 않는 아이템을 -1로 표기하는 경우 등이 있을 수 있다. 이렇게 함으로써 특별한 아이템을 처리하거나 예외를 다룰 수 있다.
all_item_count = len(item_map)
data['item'] = data['item'].apply(lambda x: item_map[x]) # remapping

In [ ]:
# user id remapping 하기
user_unique = data['user'].unique().tolist()
user_map = dict(zip(user_unique, range(1, len(user_unique) + 1)))
user_map[-1] = 0
all_user_count = len(user_map)
data['user'] = data['user'].apply(lambda x: user_map[x])

In [ ]:
# get user session
data = data.sort_values(by=['user', 'timestamps']).reset_index(drop=True) # user, timestamps 별로 정렬함
user_sessions = data.groupby('user')['item'].apply(lambda x: x.tolist()).reset_index().rename(columns={'item': 'item_list'}) # user 별로 item sequence를 얻는다.

In [ ]:
train_users = []
train_seqs = []
train_targets = []

test_users = []
test_seqs = []
test_targets = []

item_usr_clicked = {}

In [ ]:
for index, row in user_sessions.iterrows():
    user = row['user']
    items = row['item_list']

    test_item = items[-1 * 1:] # len_pred: 테스트에 사용할 아이템의 길이를 나타내는 변수
    test_seq = items[-1 * (1 + 5): -1 * 1] # len_seq: 시퀀스의 길이를 나타내는 변수
    
    test_users.append(user) # 왜 전체 user가 다 test user로 들어가지???
    test_seqs.append(test_seq)
    test_targets.append(test_item)

    train_build_items = items[:-1 * 1]

    item_usr_clicked[user] = train_build_items

    for i in range(5, len(train_build_items) - 1 + 1):
        item = train_build_items[i: i + 1] # len_tag:
        seq = train_build_items[max(0, i - 5): i]


In [ ]:
user_sessions.head()

,user,item_list
0,1,"[1, 290, 489, 381, 738, 464, 520, 11, 661, 100..."
1,2,"[61, 643, 52, 205, 365, 299, 154, 403, 50, 785..."
2,3,"[158, 96, 575, 188, 892, 322, 308, 330, 222, 8..."
3,4,"[140, 509, 495, 158, 96, 687, 404, 719, 1016, ..."
4,5,"[706, 326, 61, 158, 290, 185, 5, 96, 262, 99, ..."


In [ ]:
for index, row in user_sessions.iterrows():
    print(index)
    print()
    print(row)
    break

0

user                                                         1
item_list    [1, 290, 489, 381, 738, 464, 520, 11, 661, 100...
Name: 0, dtype: object
